## 处理Dataset

In [40]:
import pandas as pd
import numpy as np
from vmdpy import VMD  # 确保vmdpy库已经安装或者使用的是合适的VMD库
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
import pickle
from ewtpy import ewt

In [41]:
df = pd.read_csv("cleaned_station_2_flow_data1-25.csv").sort_values('time')  # 可以看出后面四列完全没用 lineID	stationID	flow_in_outlier	flow_out_outlier
df.head()

,time,flow_in,flow_out,lineID,stationID,flow_in_outlier,flow_out_outlier
0,2019-01-01 05:40:00,0.0,6.0,B,2,False,False
1,2019-01-01 05:45:00,0.0,3.0,B,2,False,False
2,2019-01-01 05:50:00,0.0,4.0,B,2,False,False
3,2019-01-01 05:55:00,0.0,7.0,B,2,False,False
4,2019-01-01 06:00:00,0.0,4.0,B,2,False,False


In [42]:
# 选择出站客流数据用于VMD分解
flow_out = df['flow_out'].values

# 标准化数据
flow_out_mean = np.mean(flow_out)
flow_out_std = np.std(flow_out)
flow_out_normalized = (flow_out - flow_out_mean) / flow_out_std

# VMD分解
alpha = 800 # moderate bandwidth constraint
tau = 0.5 # noise-tolerance (no strict fidelity enforcement)
K = 12 # - the number of modes
DC = 0  # no DC part imposed
init = 100  # initialize omegas uniformly
tol = 1e-3

u, _, _ = VMD(flow_out_normalized, alpha, tau, K, DC, init, tol)

# 创建用于存储扩展后IMFs的列表
extended_u = []

# 确保IMFs的长度与原始信号相同
for imf in u:
    if len(imf) < len(flow_out):
        # 若IMF长度小于原始信号长度，则扩展IMF
        imf = np.append(imf, np.zeros(len(flow_out) - len(imf)))
    extended_u.append(imf)

# 将扩展后的IMF列表转换成NumPy数组用于进一步处理
extended_u = np.array(extended_u)

In [43]:
pickle.dump((flow_out_mean, flow_out_std, flow_out_normalized), open('raw_data.pkl','wb'))

In [44]:
# import numpy as np
# import matplotlib.pyplot as plt
# from statsmodels.tsa.stattools import acf
# from scipy.stats import spearmanr
# from statsmodels.tsa.seasonal import seasonal_decompose
#
# # 定义检测季节性趋势的函数
# def detect_seasonality(sequence, acf_threshold=0.9, spearman_threshold=0.5):
#     # 计算序列的自相关函数
#     autocorr = acf(sequence, fft=False)
#     print(f'ACF_correlation: {max(autocorr[1:])}')
#
#     # 计算Spearman等级相关系数
#     spearman_corr, _ = spearmanr(sequence[:-1], sequence[1:])
#     print(f'Spearman correlation: {spearman_corr}')
#
#     # 判断是否存在季节性趋势
#     acf_trend = np.max(autocorr[1:]) > acf_threshold
#     spearman_trend = spearman_corr > spearman_threshold
#
#     # 如果ACF和Spearman等级相关系数都表明存在季节性趋势，则进行季节性分解
#     if acf_trend and spearman_trend:
#         # 季节性分解
#         result = seasonal_decompose(sequence, model='additive', period=220)  # 假设季节性周期为7
#         # result.plot()
#         # plt.show()
#
#         # 检查季节性分解结果中的季节性成分是否显著
#         seasonal_trend = np.max(np.abs(result.seasonal)) > 0.15 * np.max(np.abs(sequence))
#     else:
#         seasonal_trend = False
#
#     return seasonal_trend
#
# season_flag = []
# # 遍历每个序列并检测季节性趋势
# for i, sequence in enumerate(extended_u):
#     has_seasonality = detect_seasonality(sequence)
#     season_flag.append(has_seasonality)
#     if has_seasonality:
#         print(f"序列 {i+1} 存在季节性趋势")
#     else:
#         print(f"序列 {i+1} 没有季节性趋势")
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf, pacf



# 定义检测季节性趋势的函数
def detect_seasonality(sequence, acf_threshold=0.9, pacf_confint=0.1):
    # 计算序列的自相关函数和部分自相关函数
    autocorr = acf(sequence, fft=False)
    pacorr, pacorr_confint = pacf(sequence, alpha=1-pacf_confint)

    # 计算部分自相关函数的阈值
    pacf_threshold_upper = pacorr_confint[:, 1] - pacorr
    pacf_threshold_lower = pacorr - pacorr_confint[:, 0]

    # 判断是否存在季节性趋势
    acf_trend = np.any(autocorr[1:] > acf_threshold)
    pacf_trend = np.any((pacorr[1:] > pacf_threshold_upper[1:]) | (pacorr[1:] < pacf_threshold_lower[1:]))

    # 如果ACF和PACF都表明存在季节性趋势，则判断为存在季节性
    seasonal_trend = acf_trend and pacf_trend

    return seasonal_trend

season_flag = []
# 遍历每个序列并检测季节性趋势
for i, sequence in enumerate(extended_u):
    has_seasonality = detect_seasonality(sequence)
    season_flag.append(has_seasonality)
    if has_seasonality:
        print(f"序列 {i+1} 存在季节性趋势")
    else:
        print(f"序列 {i+1} 没有季节性趋势")


"""
"""


序列 1 存在季节性趋势
序列 2 存在季节性趋势
序列 3 存在季节性趋势
序列 4 存在季节性趋势
序列 5 没有季节性趋势
序列 6 没有季节性趋势
序列 7 没有季节性趋势
序列 8 存在季节性趋势
序列 9 没有季节性趋势
序列 10 没有季节性趋势
序列 11 存在季节性趋势
序列 12 没有季节性趋势


'\n'

In [37]:
subfolder = {
    True:"has_seasonality",
    False:"no_seasonality",
}

for i, seq in enumerate(extended_u):
    
    if not os.path.isdir(os.path.join("VMD_Data",subfolder[season_flag[i]])):
        os.mkdir(os.path.join("VMD_Data",subfolder[season_flag[i]]))
        
    pd.DataFrame.from_dict({
        "date":df['time'],
        "value":seq
    }).to_csv(os.path.join("VMD_Data",subfolder[season_flag[i]], "seq_{}.csv".format(i)),index=False)